# Retrieval exercise

In this exercise, you will implement the query likelihood model with Jelinek-Mercer smoothing. This assignment builds on the previous assignment for creating a Pyserini index.


## 1. Build the index
Download the MS MARCO passage collection and build an index using [Pyserini](https://github.com/castorini/pyserini).
This code is similar to PART 1 of the indexing assignment.

In [1]:
# NOTE: I comment out the pip install statements because I do them manually.
# NOTE: The file paths here may slightly alter from the original file paths.
    # This is so that the paths fit my local structure.

# !pip install pyserini
# !pip install faiss-cpu

!git clone https://github.com/castorini/anserini.git --recurse-submodules

# NOTE: Commented out because redownloading this file takes 20+ minutes.
# !wget https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz -P Data/
!tar xvfz Data/collection.tar.gz -C Data/

!cd anserini && python tools/scripts/msmarco/convert_collection_to_jsonl.py \
 --collection-path ../Data/collection.tsv --output-folder ../Data/collection_jsonl

!rm Data/*.tsv
!rm -rf sample_data

!python -m pyserini.index.lucene -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 9 \
-input Data/collection_jsonl -index Data/indexes/lucene-index-msmarco-passage -storePositions -storeDocvectors -storeRaw

Cloning into 'anserini'...
remote: Enumerating objects: 29516, done.
remote: Counting objects: 100% (3834/3834), done.
remote: Compressing objects: 100% (1065/1065), done.
remote: Total 29516 (delta 3226), reused 3175 (delta 2681), pack-reused 25682
Receiving objects: 100% (29516/29516), 85.42 MiB | 10.14 MiB/s, done.
Resolving deltas: 100% (19875/19875), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/home/daanbrugmans/projects/ru-information-retrieval-23-24/Assignments/anserini/tools'...
remote: Enumerating objects: 829, done.        
remote: Counting objects: 100% (586/586), done.        
remote: Compressing objects: 100% (498/498), done.        
remote: Total 829 (delta 114), reused 549 (delta 87), pack-reused 243        
Receiving objects: 100% (829/829), 192.26 MiB | 9.89 MiB/s, done.
Resolving deltas: 100% (198/198), done.
Submodule path 'tools': checked out '95d06f60043837a309331ffdbee7560dd1676313'
collection

## 2. Download and read the query file
You will rank MSMARCO passages for this set of queries.

In [2]:
!wget http://gem.cs.ru.nl/IR-Course/queries.txt -P Data/
    
queries = dict()
with open("Data/queries.txt", "r") as f:
    for line in f:
        cols = line.split("\t")
        queries[cols[0].strip()] = cols[1].strip()

--2023-09-27 20:03:45--  http://gem.cs.ru.nl/IR-Course/queries.txt
Resolving gem.cs.ru.nl (gem.cs.ru.nl)... 131.174.31.31
Connecting to gem.cs.ru.nl (gem.cs.ru.nl)|131.174.31.31|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2275 (2.2K) [text/plain]
Saving to: ‘Data/queries.txt’

queries.txt         100%[===================>]   2.22K  --.-KB/s    in 0s      

2023-09-27 20:03:46 (88.6 MB/s) - ‘Data/queries.txt’ saved [2275/2275]



## 3. Implement the retrieval model
You will implement language model with Jelinek-Mercer (JM) smoothing:
$$score(q,d) = \sum_{t \in q} log ((1-\lambda) \frac{c(t, d)}{|d|} + \lambda \frac{c (t, C)}{|C|}),$$
where $c (t, d)$ and $c (t, C)$ represent frequency of a term in a document and collection, respectively.

**Notes about your implementation:**
- Skip a term if it does not exist in the whole collection. This avoids $log(0)$.
- Make sure to use the right form of a query (analyzed vs. not analyzed)
- Use natural logarithm 


### 3.1. Obtain collection length
In this code, the global variable `len_C` denotes collection length.

In [3]:
from pyserini.index.lucene import IndexReader

global len_C

# =======Your code=======
index_reader = IndexReader("Data/indexes/lucene-index-msmarco-passage")
len_C = index_reader.stats()["total_terms"]
# =======================

Run this to test your code. If everything is correct, you should not get errors here. 

In [4]:
assert len_C == 352316036

### 3.2. Obtain document length

Here you need compute the length of document (as it is stored in the index).

*Hint: You first need to get the document vector from your Pyserini index. Consult [Pyseniri documentation](https://github.com/castorini/pyserini/blob/master/docs/usage-indexreader.md) to find the right function.*

In [5]:
def len_doc(d):
    # =======Your code=======
    document_vector = index_reader.get_document_vector(d)
    len_d = sum([value for _, value in document_vector.items()])
    # =======================
    return len_d  

In [6]:
# Test your code
assert len_doc("2674124") == 31

### 3.3. Obtain collection frequency of a term

Obtain number of times a term appers in the whole collection.

In [7]:
def coll_freq(t):
    # =======Your code=======
    cf = index_reader.get_term_counts(t)[1]  
    # =======================
    return cf

In [8]:
# Test your code
assert coll_freq("record") == 226439

### 3.4. Obtain term frequency

Obtain number of times a term appears in a document.

In [9]:
def term_freq(t, d):
    # =======Your code=======
    document_vector = index_reader.get_document_vector(d)
    
    if t not in document_vector:
        return 0
    
    tf = document_vector[t]
    # =======================
    return tf

In [10]:
# Test your code
assert term_freq("record", "2674124") == 2
assert term_freq("presence", "2674124") == 0

### 3.5. Compute JM-smoothed probability for a single term

Here, you need to implement the following formula:

$$P_{JM}(t,d) = (1-\lambda) \frac{c(t, d)}{|d|} + \lambda \frac{c (t, C)}{|C|}$$

In [11]:
def prob_t_Md(t, d, lambd):
    # =======Your code=======
    p_t_Md = (1 - lambd) * term_freq(t, d)/len_doc(d) + lambd * coll_freq(t)/len_C
    # =======================
    return p_t_Md

In [12]:
# Test your code
assert prob_t_Md("record", "2674124", 0.1) == 0.05812878768549357
assert prob_t_Md("darcig", "2674124", 0.1) == 0

### 3.6. Compute JM-smoothed probability for a query

In [13]:
import math

def score_doc(q, d, lambd):
    # =======Your code=======
    p_q_Md = 0
    
    for term in q:
        prob_t_Md_for_term = prob_t_Md(term, d, lambd)
        
        if prob_t_Md_for_term != 0:
            p_q_Md += math.log(prob_t_Md_for_term)
    # =======================
    return p_q_Md         

In [14]:
q1 = index_reader.analyze("are naturalization records public")
q2 = index_reader.analyze("kemeet land")
doc = "2674124"
assert score_doc(q1, doc, 0.1) == -9.227787624348021
assert score_doc(q2, doc, 0.1) == -10.254756777887694

## 4. Rank documents for the given queries
Ranking is done in two steps:
1. First pass retrieval: Use a fast ranker (i.e., Pyserini LuceneSearcher) ro rank all documents for a given query.
2. Second pass retrieval: Re-rank top-100 documents from the 1st pass retrieval using your retrieval model. This is to make the ranking process efficient.

**Notes:**
- You need to change the default values of LuceneSearcher functions to obtain top-100 documents
- Set the value of lambda to 0.1
- Store your final ranking results in the `results` variable. Every item in the `results` list is a list containing queryID, documentID, and score. This is an example how the content of results should look like:

`[['23849', '4348282', -10.65],
 ['23849', '7119957', -12.63],
 ['23849', '', -17.687729001682484], 
 ...]`

In [15]:
from pyserini.search.lucene import LuceneSearcher

results = []
searcher = LuceneSearcher("Data/indexes/lucene-index-msmarco-passage")
for qid, q in queries.items():
    # =======Your code=======
    lucene_searcher_top_100_hits = searcher.search(q, 100)
    analyzed_query = index_reader.analyze(q)
    
    for lucene_searcher_hit in lucene_searcher_top_100_hits:
        score = score_doc(analyzed_query, lucene_searcher_hit.docid, 0.1)
        results.append([qid, lucene_searcher_hit.docid, score])
    # =======================

/home/daanbrugmans/projects/ru-information-retrieval-23-24/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# Test your code
assert round(sum([item[2] for item in results]), 3) == -160109.875

Write your results into a file.
Submit this file together with the completed notebook.

In [17]:
# check duplicates
check = set()
for res in results:
    if ((res[0], res[1])) in check:
        raise Exception("Error: Duplicate query-doc is found", res[0], res[1])
    check.add((res[0], res[1]))

# write results in a file
output_str = "\n".join([l[0] + "\tQ0\t" + l[1] + "\t0\t" + str(l[2]) + "\tlm_jm" for l in results])
open("lm_jm.run", "w").write(output_str)

246890

## Handing in
Submit the result file (ranked documents), the filled-in notebook, and the pdf version of your notebook:

- The result file should be named STUDENTNUMBER_FIRSTNAME_LASTNAME_lm_jm.run
- The notebook should be named STUDENTNUMBER_FIRSTNAME_LASTNAME_retrieval.ipynb
- The pdf version of your notebook should be named STUDENTNUMBER_FIRSTNAME_LASTNAME_retrieval.pdf